In [1]:
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

# assign to first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#VRAM memory on-demand usage(prevent Keras takes all memory at the beginning)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   
sess = tf.Session(config=config)

KTF.set_session(sess)


Using TensorFlow backend.


In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

import os

batch_size = 32
num_classes = 10
epochs = 40
num_predictions = 20


(x_main, y_main), (x_test, y_test) = cifar10.load_data()

print('x shape:', x_main.shape)
x_train=x_main[:40000]
y_train=y_main[:40000]
print('x_train shape:', x_train.shape)
x_val=x_main[40000:]
y_val=y_main[40000:]
print('x_val shape:', x_val.shape)



# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_val=x_val.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_val /= 255
x_test /= 255

x shape: (50000, 32, 32, 3)
x_train shape: (40000, 32, 32, 3)
x_val shape: (10000, 32, 32, 3)


In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [4]:
mc = ModelCheckpoint('baseline_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_val, y_val),
            shuffle=True,
            callbacks=[mc])


Train on 40000 samples, validate on 10000 samples
Epoch 1/40
40000/40000 [==============================] - 9s 220us/step - loss: 1.6941 - acc: 0.3916 - val_loss: 1.4624 - val_acc: 0.4814

Epoch 00001: val_acc improved from -inf to 0.48140, saving model to baseline_model.h5
Epoch 2/40
40000/40000 [==============================] - 8s 198us/step - loss: 1.3843 - acc: 0.5076 - val_loss: 1.3373 - val_acc: 0.5274

Epoch 00002: val_acc improved from 0.48140 to 0.52740, saving model to baseline_model.h5
Epoch 3/40
40000/40000 [==============================] - 8s 196us/step - loss: 1.2365 - acc: 0.5634 - val_loss: 1.1711 - val_acc: 0.5956

Epoch 00003: val_acc improved from 0.52740 to 0.59560, saving model to baseline_model.h5
Epoch 4/40
40000/40000 [==============================] - 8s 192us/step - loss: 1.1237 - acc: 0.6056 - val_loss: 1.1089 - val_acc: 0.6165

Epoch 00004: val_acc improved from 0.59560 to 0.61650, saving model to baseline_model.h5
Epoch 5/40
40000/40000 [=================

40000/40000 [==============================] - 8s 202us/step - loss: 0.0189 - acc: 0.9947 - val_loss: 1.8898 - val_acc: 0.7327

Epoch 00040: val_acc did not improve from 0.74970


In [5]:
scores = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))
model.save('baseline_final.h5')

10000/10000 [==============================] - 0s 47us/step

Test result: 72.900 loss: 1.981


In [6]:

del model
from keras.models import load_model
model= load_model('baseline_model.h5')
scores = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 1s 50us/step

Test result: 74.200 loss: 0.954
